# Temporal Evolution Notebook Tutorial 

This is a tutorial of how does Section 5 works to help the user to understand the steps from loading data to ploting it. 

# Libraries import

In [1]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from Magics.macro import * 

from ecmwf.opendata import Client
import numpy as np
import ipyleaflet
from ipyleaflet import Map, Marker

import ipywidgets
import xarray as xr
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt

from bqplot import Lines, Figure, LinearScale, Axis, DateScale
import bqplot
from datetime import datetime

# Import utilities functions from TropiDash main folder
import os
cwd = os.getcwd()
cwd = os.path.sep.join(cwd.split(os.path.sep)[:-1])
import sys
sys.path.insert(1, os.path.join(cwd, 'TropiDash'))
import utils_tracks as tracks

from utils_TemporalEvolution import *
from utils_atm import *

# Download data (it's done in sections 1 and 2)

In [2]:
# S1
start_date = datetime(2023, 9, 7, 0, 0)

start_date_forecast = start_date
# Load the ECMWF forecast data
df_storms_forecast = tracks.create_storms_df(start_date)

# Select the data of storm LEE (Storm Identifier: 13L)
df_storm_forecast = df_storms_forecast[df_storms_forecast.stormIdentifier == '13L']


locations_avg, timesteps_avg, pressures_avg, wind_speeds_avg = tracks.mean_forecast_track(df_storm_forecast)

In [3]:
#steps to download
cyclone_days = 2
stdate =  datetime(2023, 9, 7, 0, 0)
windprobsteps = [f"{12 * i}-{ 12 * i + 24}" for i in range(2*cyclone_days-1)]
windprobsteps.insert(0, "0-24")
stepsdict = {
               "base": list(np.arange(12, 240, 12)[0:2*cyclone_days]),
               "10fgg25": windprobsteps
            }

variables = ['msl', 'skt', 'tp', '10fgg25'] #list of variables to visualize


# Download
fnames = dwnl_atmdata_step(variables, stepsdict, stdate);
# Load
vardict = load_atmdata(variables, fnames)



In [4]:
initial_latlon = locations_avg[0] #(38, -90) #first cyclone position
steps_to_download = stepsdict['base']

steps_to_download2 = stepsdict['10fgg25']

initial_date = start_date.strftime("%Y%m%d")


# Load data

In [5]:
### LOAD DATA
data_allpoints = get_allvars_allpoints(initial_date, steps_to_download, steps_to_download2)
data_allpoints['skt']['Value'] = data_allpoints['skt']['Value'] - 273.15
data_allpoints['msl']['Value'] = data_allpoints['msl']['Value']/100
#print('Printing map...')
    


# Create background map

In [6]:
m = Map(
    center=[initial_latlon[0], initial_latlon[1]+50],
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    zoom = 2,
)

  

# Create plots of the initial marker location

In [7]:
p1, p2, p3, p4 = get_initial_plot(data_allpoints, initial_date, initial_latlon, steps_to_download)
    

# Transform the plots into a widget from the plot

In [10]:
item_layout = ipywidgets.Layout(overflow_y='scroll', width='350px', height='350px',
                               flex_flow='column', display='block')
main_figure = widgets.Box(children=[p1, p2, p3, p4], layout=item_layout,  width='350px', height='350px')

widget_control1 = ipyleaflet.WidgetControl(widget=main_figure, position="bottomright")
m.add(widget_control1)

 

Map(center=[15.150012821314707, 2.9538448747324253], controls=(ZoomControl(options=['position', 'zoom_in_text'…

# Add a marker to the background map

In [11]:
marker = Marker(location=initial_latlon, draggable=True, name = 'Position') 
m.add_layer(marker)
marker.on_move(handle_move(data_allpoints, steps_to_download))


In [12]:
avg_track = locations_avg


In [13]:
avg_track_antpath = ipyleaflet.AntPath(locations = avg_track, color = "red")

m.add_layer(avg_track_antpath)


In [14]:
display(m)

Map(bottom=668.0, center=[15.150012821314707, 2.9538448747324253], controls=(ZoomControl(options=['position', …